In [1]:
#from numpy.random import seed
#seed(42)
#import tensorflow
#tensorflow.random.set_seed(42)
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0
# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED']=str(seed_value)
#os.environ['CUDA_VISIBLE_DEVICES']= "-1"
# 2. Set the `python` built-in pseudo-random generator at a fixed value
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)
import random
random.seed(seed_value)
# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)
# 4. Set the `tensorflow` pseudo-random generator at a fixed value

tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)
# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess =tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

2021-09-01 19:39:23.429442: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-09-01 19:39:24.032087: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-09-01 19:39:24.075906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-01 19:39:24.076474: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.755GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2021-09-01 19:39:24.076491: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-09-01 19:39:24.078409: I tensorflow/stream_executor/

In [2]:
%matplotlib notebook
import numpy as np
import pandas as pd
from data_extraction import *
from resp_signal_extraction import *
from rr_extration import *
from sklearn.preprocessing import MinMaxScaler
import re
import pickle as pkl
from tf_model import *
from tf_model_evi import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
import evidential_deep_learning as edl
import matplotlib.pyplot as plt
from filters import *
import plotly as py
#import plotly.figure_factory as ff
import ipywidgets as widgets
import plotly.graph_objs as go
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#py.offline.init_notebook_mode(connected = True)
#from plotly import tools
import plotly.express as px

In [3]:
input_conf = 'confd'

In [4]:
with open('output','rb') as f:
    output_data = pkl.load(f)

with open('input','rb') as f:
    input_data = pkl.load(f)

with open('raw_signal.pkl','rb') as f:
    raw_data = pkl.load(f)

input_data = np.transpose(input_data, (0,2,1))
raw_data = np.transpose(raw_data, (0,2,1))
annotation = pd.read_pickle('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/annotation.pkl')
reference_rr = (annotation['Reference_RR'].values).reshape(-1,1)

input_data = np.around(input_data , decimals = 4)
raw_data = np.around(raw_data , decimals = 4)
output_data = np.around(output_data , decimals = 4)
reference_rr = np.around(reference_rr , decimals = 4)

tensor_input = tf.convert_to_tensor(input_data, dtype = 'float32')
tensor_output = tf.convert_to_tensor(output_data, dtype = 'float32')
tensor_ref_rr = tf.convert_to_tensor(reference_rr, dtype = 'float32')
tensor_raw_data = tf.convert_to_tensor(raw_data, dtype = 'float32')
training_ids = annotation['patient_id'] < 13

x_train_data = tensor_input[tf.convert_to_tensor(training_ids.values)]
x_test_data = tensor_input[tf.convert_to_tensor(~(training_ids.values))]
x_train_ref_rr = tensor_ref_rr[tf.convert_to_tensor(training_ids.values)]
x_test_ref_rr = tensor_ref_rr[tf.convert_to_tensor(~(training_ids.values))]
x_train_raw_sig = tensor_raw_data[tf.convert_to_tensor(training_ids.values)]
x_test_raw_sig = tensor_raw_data[tf.convert_to_tensor(~(training_ids.values))]

y_train_data = tensor_output[tf.convert_to_tensor(training_ids.values)]
y_test_data = tensor_output[tf.convert_to_tensor(~(training_ids.values))]


2021-09-01 19:39:25.984501: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-01 19:39:25.984932: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.755GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2021-09-01 19:39:25.984973: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-01 19:39:25.985335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-01 19:39:25.985672: I tensorflow/core/comm

In [5]:
if input_conf == 'confc':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data , y_train_data))
        train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data , y_test_data))
        test_dataset = test_dataset.batch(128)

if input_conf == 'confb':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data , x_train_ref_rr))
        train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data , x_test_ref_rr))
        test_dataset = test_dataset.batch(128)

if input_conf == 'confd':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data , y_train_data, x_train_ref_rr))
        train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data , y_test_data, x_test_ref_rr))
        test_dataset = test_dataset.batch(128)
if input_conf == 'confa':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_raw_sig , x_train_ref_rr))
        train_dataset = train_dataset.shuffle(len(x_train_raw_sig)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_raw_sig , x_test_ref_rr))
        test_dataset = test_dataset.batch(128)

if input_conf == 'confe':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_raw_sig , y_train_data))
        train_dataset = train_dataset.shuffle(len(x_train_raw_sig)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_raw_sig , y_test_data))
        test_dataset = test_dataset.batch(128)
if input_conf == 'conff':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_raw_sig , y_train_data, x_train_ref_rr))
        train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_raw_sig , y_test_data, x_test_ref_rr))
        test_dataset = test_dataset.batch(128)
    

In [6]:
if input_conf == 'confc':
    model_input_shape = (128,3)
    model  = BRUnet(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/confc/best_model_100.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step,(x_batch_test , y_batch_test) in enumerate(test_dataset):
        output = model(x_batch_test)
        test_loss = loss_fn(y_batch_test,output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
#-----------------------------------------------------------------------------------------------------------------
if input_conf == 'confd':
    model_input_shape = (128,3)
    model  = BRUnet_Multi_resp(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/confd/best_model_100.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    final_output_rr = tf.convert_to_tensor([])
    for step , (x_batch_test,y_batch_test,x_batch_test_ref_rr) in enumerate(test_dataset):
        test_output,test_out_rr = model(x_batch_test)
        test_loss_resp = loss_fn(y_batch_test  , test_output)
        test_loss_rr = loss_fn(x_batch_test_ref_rr , test_out_rr)
        test_loss = test_loss_resp + test_loss_rr
        if step == 0:
            final_output = test_output
            final_output_rr = test_out_rr
        else:
            final_output = tf.concat([final_output , test_output] , axis = 0)
            final_output_rr = tf.concat([final_output_rr , test_out_rr] , axis = 0)
        test_loss_list.append(test_loss)
            
#------------------------------------------------------------------------------------------------------------------
if input_conf == 'confb':
    model_input_shape = (128,3)
    model  = BRUnet_Encoder(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/confb/best_model_100.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test,x_batch_test_ref_rr) in enumerate(test_dataset):
        output = model(x_batch_test)
        test_loss = loss_fn(x_batch_test_ref_rr , output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
#--------------------------------------------------------------------------------------------------------------------
if input_conf == 'confe':
    model_input_shape = (2048,3)
    model  = BRUnet_raw(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/confe/best_model_100.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test_raw,y_batch_test) in enumerate(test_dataset):
        output = model(x_batch_test_raw)
        test_loss = loss_fn(y_batch_test , output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
        
#-------------------------------------------------------------------------------------------------------------------
if input_conf == 'confa':
    model_input_shape = (2048,3)
    model  = BRUnet_raw_encoder(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/confa/best_model_100.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test_raw, x_batch_test_ref_rr) in enumerate(test_dataset):
        output = model(x_batch_test_raw)
        test_loss = loss_fn(x_batch_test_ref_rr , output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
#-------------------------------------------------------------------------------------------------------------------
if input_conf == 'conff':
    model_input_shape = (2048,3)
    model  = BRUnet_raw_multi(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/conff/best_model_100.h5')        
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    final_output_rr = tf.convert_to_tensor([])
    for step , (x_batch_test_raw , y_batch_test , x_batch_test_ref_rr) in enumerate(test_dataset):
        test_output,test_out_rr = model(x_batch_test_raw)
        test_loss_resp = loss_fn(y_batch_test  , test_output)
        test_loss_rr = loss_fn(x_batch_test_ref_rr , test_out_rr)
        test_loss = test_loss_resp + test_loss_rr
        if step == 0:
            final_output = test_output
            final_output_rr = test_out_rr
        else:
            final_output = tf.concat([final_output , test_output] , axis = 0)
            final_output_rr = tf.concat([final_output_rr , test_out_rr] , axis = 0)
        test_loss_list.append(test_loss)   

2021-09-01 19:39:26.631013: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-09-01 19:39:27.444778: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101
2021-09-01 19:39:28.292469: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-09-01 19:39:29.297018: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-09-01 19:39:29.312755: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [7]:
def extremas_extraction(signal):
    avg_breath_duration = np.array([])
    extrema_relevent = []
    for item in signal:
        amplitude = np.array([])
        pos_peaks , _ = scipy.signal.find_peaks(item , height = [-3000,3000])
        neg_peaks , _ = scipy.signal.find_peaks(-1*item , height = [-3000 , 3000])
        extremas = np.concatenate((pos_peaks , neg_peaks))
        extremas = np.sort(extremas)
        for i in range(len(extremas)):
            amplitude = np.append(amplitude , item[int(extremas[i])])
        amplitude_diff = np.abs(np.diff(amplitude))
        q3 = np.percentile(amplitude_diff , 75)
        threshold = 0.3*q3
        eliminate_pairs_of_extrema = 1
        while(eliminate_pairs_of_extrema):
            amps = np.array([])
            if len(extremas)<3:
                eliminate_pairs_of_extrema = 0
                continue
            for i in range(len(extremas)):
                amps = np.append(amps , item[int(extremas[i])])
            amp_diff = np.abs(np.diff(amps)) 
            min_amp_diff , index = min(amp_diff) , np.argmin(amp_diff)
            #print(min_amp_diff)
            if min_amp_diff > threshold:
                eliminate_pairs_of_extrema = 0
                #extrema_relevent = extremas
            else:
                extremas = np.concatenate((extremas[0:index] , extremas[index+2 :]))
                #amplitude_diff = np.delete(amplitude_diff , index)
        if item[int(extremas[0])] < item[int(extremas[1])]:
            extremas = extremas[1:]
        if item[int(extremas[-1])] < item[int(extremas[-2])]:
            extremas = extremas[:-1]
        no_of_breaths = (len(extremas)-1)/2
        breath_duration = extremas[-1] - extremas[0]
        avg_breath_duration = np.append(avg_breath_duration , breath_duration/no_of_breaths)
        extrema_relevent.append(extremas)
    return avg_breath_duration , extrema_relevent

In [8]:
ref_sig = y_test_data.numpy()
fbpB , fbpA = band_pass(0.1,0.8,8)
final_ref_resp_sig = []
for item in ref_sig:
    final_ref_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
final_ref_resp_sig = np.array(final_ref_resp_sig)

In [9]:
if input_conf == 'confa':
    final_output_rr = final_output.numpy()
    final_output_rr = final_output_rr.reshape(final_output_rr.shape[0],final_output_rr.shape[1])
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    error = np.abs(avg_ref_breath - final_output_rr)
    mae = np.mean(error)
    rmse = np.sqrt(np.mean(error**2))
    print('Mean Absolute Error for {} is: {}'.format(input_conf,mae))
    print('Root Mean Square Error for {} is: {}'.format(input_conf,rmse))
#----------------------------------------------------------------------------------------------------------------   
if input_conf == 'confb':
    final_output_rr = final_output.numpy()
    final_output_rr = final_output_rr.reshape(final_output_rr.shape[0],final_output_rr.shape[1])
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    error = np.abs(avg_ref_breath - final_output_rr)
    mae = np.mean(error)
    rmse = np.sqrt(np.mean(error**2))
    print('Mean Absolute Error for {} is: {}'.format(input_conf,mae))
    print('Root Mean Square Error for {} is: {}'.format(input_conf,rmse))
    

In [10]:
#Mean Absolute Error for confa is: 2.5189870106871095
#Root Mean Square Error for confa is: 3.1476790448298395

In [11]:
if input_conf == 'confc':
    final_output_resp_sig = []
    inst_br_dur = []
    inst_ref_br_dur = []
    inst_rr = []
    inst_ref_rr = []
    
    final_output_resp = final_output.numpy()
    final_output_resp = final_output_resp.reshape(final_output_resp.shape[0],final_output_resp.shape[1])
    for item in final_output_resp:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_breaths = (60*4/duration_resp).reshape(-1,1)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - avg_breaths)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    

    for item in extremas_resp:
        inst_br_dur.append(np.diff(item[0::2]))
    for item in extremas_ref_resp:
        inst_ref_br_dur.append(np.diff(item[0::2]))
    for item in inst_br_dur:
        inst_rr.append(np.mean(60*4/item))
    for item in inst_ref_br_dur:
        inst_ref_rr.append(np.mean(60*4/item))
    
    inst_rr = (np.array(inst_rr)).reshape(-1,1)
    inst_ref_rr = (np.array(inst_ref_rr)).reshape(-1,1)
    
    error_inst_breaths = np.abs(inst_ref_rr - inst_rr)
    mae_inst_breath = np.mean(error_inst_breaths)
    rmse_inst_breath = np.sqrt(np.mean(error_inst_breaths**2))
    
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))
    
    print('Mean Absolute Error instantaneous wise for {} is: {}'.format(input_conf,mae_inst_breath))
    print('Root Mean Square Error instantaneous wise for {} is: {}'.format(input_conf,rmse_inst_breath))

#--------------------------------------------------------------------------------------------------------------------    
if input_conf == 'confe':
    final_output_resp_sig = []
    inst_br_dur = []
    inst_ref_br_dur = []
    inst_rr = []
    inst_ref_rr = []
    final_output_resp = final_output.numpy()
    final_output_resp = final_output_resp.reshape(final_output_resp.shape[0],final_output_resp.shape[1])
    for item in final_output_resp:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_breaths = (60*4/duration_resp).reshape(-1,1)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - avg_breaths)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    
    for item in extremas_resp:
        inst_br_dur.append(np.diff(item[0::2]))
    for item in extremas_ref_resp:
        inst_ref_br_dur.append(np.diff(item[0::2]))
    for item in inst_br_dur:
        inst_rr.append(np.mean(60*4/item))
    for item in inst_ref_br_dur:
        inst_ref_rr.append(np.mean(60*4/item))
    
    inst_rr = (np.array(inst_rr)).reshape(-1,1)
    inst_ref_rr = (np.array(inst_ref_rr)).reshape(-1,1)
    
    error_inst_breaths = np.abs(inst_ref_rr - inst_rr)
    mae_inst_breath = np.mean(error_inst_breaths)
    rmse_inst_breath = np.sqrt(np.mean(error_inst_breaths**2))
    
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))
    
    print('Mean Absolute Error instantaneous wise for {} is: {}'.format(input_conf,mae_inst_breath))
    print('Root Mean Square Error instantaneous wise for {} is: {}'.format(input_conf,rmse_inst_breath))
        
    

In [12]:
#Mean Absolute Error average wise for confc is: 2.993400011948415
#Root Mean Square Error average wise for confc is: 3.7908212294622583
#Mean Absolute Error instantaneous wise for confc is: 3.097192353299057
#Root Mean Square Error instantaneous wise for confc is: 3.927976655516534

In [13]:
if input_conf == 'confd':
    final_output_resp_sig = []
    inst_br_dur = []
    inst_ref_br_dur = []
    inst_rr = []
    inst_ref_rr = []
    
    final_output_resp = final_output.numpy()
    final_rr = final_output_rr.numpy()
    final_output_resp = final_output_resp.reshape(final_output_resp.shape[0],final_output_resp.shape[1])
    final_rr = final_rr.reshape(final_rr.shape[0],final_rr.shape[1])
    for item in final_output_resp:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
        
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - final_rr)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    
    for item in extremas_resp:
        inst_br_dur.append(np.diff(item[0::2]))
    for item in extremas_ref_resp:
        inst_ref_br_dur.append(np.diff(item[0::2]))
    for item in inst_br_dur:
        inst_rr.append(np.mean(60*4/item))
    for item in inst_ref_br_dur:
        inst_ref_rr.append(np.mean(60*4/item))
    
    inst_rr = (np.array(inst_rr)).reshape(-1,1)
    inst_ref_rr = (np.array(inst_ref_rr)).reshape(-1,1)
    
    error_inst_breaths = np.abs(inst_ref_rr - inst_rr)
    mae_inst_breath = np.mean(error_inst_breaths)
    rmse_inst_breath = np.sqrt(np.mean(error_inst_breaths**2))
    
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))
    
    print('Mean Absolute Error instantaneous wise for {} is: {}'.format(input_conf,mae_inst_breath))
    print('Root Mean Square Error instantaneous wise for {} is: {}'.format(input_conf,rmse_inst_breath))

#-------------------------------------------------------------------------------------------------------
if input_conf == 'conff':
    final_output_resp_sig = []
    inst_br_dur = []
    inst_ref_br_dur = []
    inst_rr = []
    inst_ref_rr = []
    
    final_output_resp = final_output.numpy()
    final_rr = final_output_rr.numpy()
    final_output_resp = final_output_resp.reshape(final_output_resp.shape[0],final_output_resp.shape[1])
    final_rr = final_rr.reshape(final_rr.shape[0],final_rr.shape[1])
    for item in final_output_resp:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
        
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - final_rr)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    
    for item in extremas_resp:
        inst_br_dur.append(np.diff(item[0::2]))
    for item in extremas_ref_resp:
        inst_ref_br_dur.append(np.diff(item[0::2]))
    for item in inst_br_dur:
        inst_rr.append(np.mean(60*4/item))
    for item in inst_ref_br_dur:
        inst_ref_rr.append(np.mean(60*4/item))
    
    inst_rr = (np.array(inst_rr)).reshape(-1,1)
    inst_ref_rr = (np.array(inst_ref_rr)).reshape(-1,1)
    
    error_inst_breaths = np.abs(inst_ref_rr - inst_rr)
    mae_inst_breath = np.mean(error_inst_breaths)
    rmse_inst_breath = np.sqrt(np.mean(error_inst_breaths**2))
    
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))
    
    print('Mean Absolute Error instantaneous wise for {} is: {}'.format(input_conf,mae_inst_breath))
    print('Root Mean Square Error instantaneous wise for {} is: {}'.format(input_conf,rmse_inst_breath))



Mean Absolute Error average wise for confd is: 2.392965419428223
Root Mean Square Error average wise for confd is: 3.0249466151830364
Mean Absolute Error instantaneous wise for confd is: 2.971450391669577
Root Mean Square Error instantaneous wise for confd is: 3.720807591978334


# WITH EVIDENTIAL

In [14]:
if input_conf == 'confc':
    model_input_shape = (128,3)
    model  = BRUnet_evi(model_input_shape)
    coeff_val = 0.01
    #loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODEL_WITH_EVI/confc/best_model__coeff_0.01.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step,(x_batch_test , y_batch_test) in enumerate(test_dataset):
        y_batch_test = tf.expand_dims(y_batch_test , axis = -1)
        output = model(x_batch_test)
        test_loss = edl.losses.EvidentialRegression(y_batch_test,output, coeff = coeff_val)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
#------------------------------------------------------------------------------------------------------------------------
if input_conf == 'confe':
    model_input_shape = (2048,3)
    model  = BRUnet_raw_evi(model_input_shape)
    coeff_val = 0.01
    #loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODEL_WITH_EVI/confe/best_model___coeff_0.01.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test_raw,y_batch_test) in enumerate(test_dataset):
        y_batch_test = tf.expand_dims(y_batch_test , axis = -1)
        output = model(x_batch_test_raw)
        test_loss = edl.losses.EvidentialRegression(y_batch_test,output, coeff = coeff_val)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
#------------------------------------------------------------------------------------------------------------------
if input_conf == 'confb':
    model_input_shape = (128,3)
    model  = BRUnet_Encoder_evi(model_input_shape)
    coeff_val = 0.005
    #loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODEL_WITH_EVI/confb/best_model___coeff_0.005.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test,x_batch_test_ref_rr) in enumerate(test_dataset):
        x_batch_test_ref_rr = tf.expand_dims(x_batch_test_ref_rr , axis = -1)
        output = model(x_batch_test)
        test_loss = edl.losses.EvidentialRegression(x_batch_test_ref_rr , output , coeff = coeff_val)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
    

#--------------------------------------------------------------------------------------------------------------
if input_conf == 'confa':
    model_input_shape = (2048,3)
    model  = BRUnet_raw_encoder_evi(model_input_shape)
    #loss_fn = Huber()
    coeff_val = 0.01
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODEL_WITH_EVI/confa/best_model___coeff_0.01.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test_raw, x_batch_test_ref_rr) in enumerate(test_dataset):
        x_batch_test_ref_rr = tf.expand_dims(x_batch_test_ref_rr , axis = -1)
        output = model(x_batch_test_raw)
        test_loss = edl.losses.EvidentialRegression(x_batch_test_ref_rr , output, coeff = coeff_val)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)  
        
#------------------------------------------------------------------------------------------------------------------------
if input_conf == 'conff':
    model_input_shape = (2048,3)
    model  = BRUnet_raw_multi_evi(model_input_shape)
    #loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    coeff_val = 0.05
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODEL_WITH_EVI/conff/best_model___coeff_0.05.h5')        
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    final_output_rr = tf.convert_to_tensor([])
    for step , (x_batch_test_raw , y_batch_test , x_batch_test_ref_rr) in enumerate(test_dataset):
        y_batch_test = tf.expand_dims(y_batch_test , axis = -1)
        x_batch_test_ref_rr = tf.expand_dims(x_batch_test_ref_rr , axis = -1)
        test_output,test_out_rr = model(x_batch_test_raw)
        test_loss_resp = edl.losses.EvidentialRegression(y_batch_test  , test_output , coeff = coeff_val)
        test_loss_rr = edl.losses.EvidentialRegression(x_batch_test_ref_rr , test_out_rr , coeff = coeff_val)
        test_loss = test_loss_resp + test_loss_rr
        if step == 0:
            final_output = test_output
            final_output_rr = test_out_rr
        else:
            final_output = tf.concat([final_output , test_output] , axis = 0)
            final_output_rr = tf.concat([final_output_rr , test_out_rr] , axis = 0)
        test_loss_list.append(test_loss) 
#-------------------------------------------------------------------------------------------------------------------------
if input_conf == 'confd':
    model_input_shape = (128,3)
    model  = BRUnet_Multi_resp_evi(model_input_shape)
    #loss_fn = Huber()
    model(tf.ones((128,128,3)))
    coeff_val = 0.005
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODEL_WITH_EVI/confd/best_model___coeff_0.005.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    final_output_rr = tf.convert_to_tensor([])
    for step , (x_batch_test,y_batch_test,x_batch_test_ref_rr) in enumerate(test_dataset):
        x_batch_test_ref_rr = tf.expand_dims(x_batch_test_ref_rr , axis = -1)
        y_batch_test = tf.expand_dims(y_batch_test , axis = -1)
        test_output,test_out_rr = model(x_batch_test)
        test_loss_resp = edl.losses.EvidentialRegression(y_batch_test  , test_output, coeff = coeff_val)
        test_loss_rr = edl.losses.EvidentialRegression(x_batch_test_ref_rr , test_out_rr,coeff = coeff_val)
        test_loss = test_loss_resp + test_loss_rr
        if step == 0:
            final_output = test_output
            final_output_rr = test_out_rr
        else:
            final_output = tf.concat([final_output , test_output] , axis = 0)
            final_output_rr = tf.concat([final_output_rr , test_out_rr] , axis = 0)
        test_loss_list.append(test_loss)

In [15]:
gamma, lamda, alpha, beta = tf.split(final_output, 4, axis=-1)
output = gamma.numpy()
output = output.reshape(output.shape[0] , output.shape[1])


In [16]:
if input_conf == 'confc':
    final_output_resp_sig = []
    gamma, lamda, alpha, beta = tf.split(final_output, 4, axis=-1)
    output = gamma.numpy()
    output = output.reshape(output.shape[0] , output.shape[1])
    for item in output:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_breaths = (60*4/duration_resp).reshape(-1,1)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - avg_breaths)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    
    final_lamda = lamda.numpy()
    final_alpha = alpha.numpy()
    final_beta = beta.numpy()
    
    final_lamda = final_lamda.reshape(final_lamda.shape[0],final_lamda.shape[1])
    final_alpha = final_alpha.reshape(final_alpha.shape[0],final_alpha.shape[1])
    final_beta = final_beta.reshape(final_beta.shape[0],final_beta.shape[1])
    #x_samps = np.arange(0,len(y_train_data[0]))
    aleatoric = final_beta/(final_alpha - 1)
    epistemic = final_beta/(final_lamda*(final_alpha - 1))
    std = np.sqrt(epistemic)
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))
#-------------------------------------------------------------------------------------------------------
if input_conf == 'confe':
    final_output_resp_sig = []
    gamma, lamda, alpha, beta = tf.split(final_output, 4, axis=-1)
    output = gamma.numpy()
    output = output.reshape(output.shape[0] , output.shape[1])
    for item in output:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_breaths = (60*4/duration_resp).reshape(-1,1)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - avg_breaths)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    
    final_lamda = lamda.numpy()
    final_alpha = alpha.numpy()
    final_beta = beta.numpy()
    
    final_lamda = final_lamda.reshape(final_lamda.shape[0],final_lamda.shape[1])
    final_alpha = final_alpha.reshape(final_alpha.shape[0],final_alpha.shape[1])
    final_beta = final_beta.reshape(final_beta.shape[0],final_beta.shape[1])
    x_samps = np.arange(0,len(y_train_data[0]))
    aleatoric = final_beta/(final_alpha - 1)
    epistemic = final_beta/(final_lamda*(final_alpha - 1))
    
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))

#---------------------------------------------------------------------------------------------------------
if input_conf == 'confb':
    gamma, lamda, alpha, beta = tf.split(final_output, 4, axis=-1)
    final_output_rr = gamma.numpy()
    final_output_rr = final_output_rr.reshape(final_output_rr.shape[0],final_output_rr.shape[1])
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    error = np.abs(avg_ref_breath - final_output_rr)
    mae = np.mean(error)
    rmse = np.sqrt(np.mean(error**2))
    print('Mean Absolute Error for {} is: {}'.format(input_conf,mae))
    print('Root Mean Square Error for {} is: {}'.format(input_conf,rmse))
    
    final_lamda = lamda.numpy()
    final_alpha = alpha.numpy()
    final_beta = beta.numpy()
    
    final_lamda = final_lamda.reshape(final_lamda.shape[0],final_lamda.shape[1])
    final_alpha = final_alpha.reshape(final_alpha.shape[0],final_alpha.shape[1])
    final_beta = final_beta.reshape(final_beta.shape[0],final_beta.shape[1])
    #x_samps = np.arange(0,len(y_train_data[0]))
    aleatoric = final_beta/(final_alpha - 1)
    epistemic = final_beta/(final_lamda*(final_alpha - 1))
    print(epistemic)
    samples = np.arange(0,len(final_output_rr)).reshape(-1,1)
    var = np.sqrt(epistemic)
    aleatoric = aleatoric/max(aleatoric)
    epistemic = epistemic/max(epistemic)
    confb_data = np.hstack((samples,final_output_rr , epistemic , aleatoric))
    col_confb = ['Samples','Final RR Output (BrPM)' , 'Epistemic','Aleatoric']
    data_confb = pd.DataFrame(confb_data , columns = col_confb)
    

#------------------------------------------------------------------------------------------------------
if input_conf == 'confa':
    gamma, lamda, alpha, beta = tf.split(final_output, 4, axis=-1)
    final_output_rr = gamma.numpy()
    final_output_rr = final_output_rr.reshape(final_output_rr.shape[0],final_output_rr.shape[1])
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    error = np.abs(avg_ref_breath - final_output_rr)
    mae = np.mean(error)
    rmse = np.sqrt(np.mean(error**2))
    print('Mean Absolute Error for {} is: {}'.format(input_conf,mae))
    print('Root Mean Square Error for {} is: {}'.format(input_conf,rmse))
    
    final_lamda = lamda.numpy()
    final_alpha = alpha.numpy()
    final_beta = beta.numpy()
    
    final_lamda = final_lamda.reshape(final_lamda.shape[0],final_lamda.shape[1])
    final_alpha = final_alpha.reshape(final_alpha.shape[0],final_alpha.shape[1])
    final_beta = final_beta.reshape(final_beta.shape[0],final_beta.shape[1])
    aleatoric = final_beta/(final_alpha - 1)
    epistemic = final_beta/(final_lamda*(final_alpha - 1))
    print(epistemic)
    samples = np.arange(0,len(final_output_rr)).reshape(-1,1)
    aleatoric = aleatoric/max(aleatoric)
    epistemic = epistemic/max(epistemic)
    confa_data = np.hstack((samples,final_output_rr , epistemic , aleatoric))
    col_confa = ['Samples','Final RR Output (BrPM)' , 'Epistemic','Aleatoric']
    data_confa = pd.DataFrame(confa_data , columns = col_confa)
#-----------------------------------------------------------------------------------------------------
if input_conf == 'conff':
    final_output_resp_sig = []
    inst_br_dur = []
    inst_ref_br_dur = []
    inst_rr = []
    inst_ref_rr = []
    
    gamma_resp, lamda_resp, alpha_resp, beta_resp = tf.split(final_output, 4, axis=-1)
    gamma_rr, lamda_rr, alpha_rr, beta_rr = tf.split(final_output_rr, 4, axis=-1)
    final_output_resp = gamma_resp.numpy()
    final_rr = gamma_rr.numpy()
    final_output_resp = final_output_resp.reshape(final_output_resp.shape[0] , final_output_resp.shape[1])
    final_rr = final_rr.reshape(final_rr.shape[0],final_rr.shape[1])
    for item in final_output_resp:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
    
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - final_rr)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    
    for item in extremas_resp:
        inst_br_dur.append(np.diff(item[0::2]))
    for item in extremas_ref_resp:
        inst_ref_br_dur.append(np.diff(item[0::2]))
    for item in inst_br_dur:
        inst_rr.append(np.mean(60*4/item))
    for item in inst_ref_br_dur:
        inst_ref_rr.append(np.mean(60*4/item))
    
    inst_rr = (np.array(inst_rr)).reshape(-1,1)
    inst_ref_rr = (np.array(inst_ref_rr)).reshape(-1,1)
    
    error_inst_breaths = np.abs(inst_ref_rr - inst_rr)
    mae_inst_breath = np.mean(error_inst_breaths)
    rmse_inst_breath = np.sqrt(np.mean(error_inst_breaths**2))
    
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))
    
    print('Mean Absolute Error instantaneous wise for {} is: {}'.format(input_conf,mae_inst_breath))
    print('Root Mean Square Error instantaneous wise for {} is: {}'.format(input_conf,rmse_inst_breath))
    
    final_lamda_resp = lamda_resp.numpy()
    final_alpha_resp = alpha_resp.numpy()
    final_beta_resp = beta_resp.numpy()
    
    final_lamda_rr = lamda_rr.numpy()
    final_alpha_rr = alpha_rr.numpy()
    final_beta_rr = beta_rr.numpy()
    
    aleatoric = final_beta_resp/(final_alpha_resp - 1)
    epistemic = final_beta_resp/(final_lamda_resp*(final_alpha_resp - 1))
    
    aleatoric_rr = final_beta_rr/(final_alpha_rr - 1)
    epistemic_rr = final_beta_rr/(final_lamda_rr*(final_alpha_rr - 1))
    print(epistemic_rr)
    samples = np.arange(0,len(final_rr)).reshape(-1,1)
    epistemic_rr = epistemic_rr.reshape(epistemic_rr.shape[0],epistemic_rr.shape[1])
    aleatoric_rr = aleatoric_rr.reshape(aleatoric_rr.shape[0],aleatoric_rr.shape[1])
    epistemic_rr = epistemic_rr/max(epistemic_rr)
    aleatoric_rr = aleatoric_rr/max(aleatoric_rr)
    
    conff_data = np.hstack((samples,final_rr , epistemic_rr , aleatoric_rr))
    col_conff = ['Samples','Final RR Output (BrPM)' , 'Epistemic','Aleatoric']
    data_conff = pd.DataFrame(conff_data , columns = col_conff)
    
#----------------------------------------------------------------------------------------------------------------
if input_conf == 'confd':
    final_output_resp_sig = []
    inst_br_dur = []
    inst_ref_br_dur = []
    inst_rr = []
    inst_ref_rr = []
    
    gamma_resp, lamda_resp, alpha_resp, beta_resp = tf.split(final_output, 4, axis=-1)
    gamma_rr, lamda_rr, alpha_rr, beta_rr = tf.split(final_output_rr, 4, axis=-1)
    final_output_resp = gamma_resp.numpy()
    final_rr = gamma_rr.numpy()
    final_output_resp = final_output_resp.reshape(final_output_resp.shape[0] , final_output_resp.shape[1])
    final_rr = final_rr.reshape(final_rr.shape[0],final_rr.shape[1])
    for item in final_output_resp:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
    
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - final_rr)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    
    for item in extremas_resp:
        inst_br_dur.append(np.diff(item[0::2]))
    for item in extremas_ref_resp:
        inst_ref_br_dur.append(np.diff(item[0::2]))
    for item in inst_br_dur:
        inst_rr.append(np.mean(60*4/item))
    for item in inst_ref_br_dur:
        inst_ref_rr.append(np.mean(60*4/item))
    
    inst_rr = (np.array(inst_rr)).reshape(-1,1)
    inst_ref_rr = (np.array(inst_ref_rr)).reshape(-1,1)
    
    error_inst_breaths = np.abs(inst_ref_rr - inst_rr)
    mae_inst_breath = np.mean(error_inst_breaths)
    rmse_inst_breath = np.sqrt(np.mean(error_inst_breaths**2))
    
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))
    
    print('Mean Absolute Error instantaneous wise for {} is: {}'.format(input_conf,mae_inst_breath))
    print('Root Mean Square Error instantaneous wise for {} is: {}'.format(input_conf,rmse_inst_breath))
    
    final_lamda_resp = lamda_resp.numpy()
    final_alpha_resp = alpha_resp.numpy()
    final_beta_resp = beta_resp.numpy()
    
    final_lamda_rr = lamda_rr.numpy()
    final_alpha_rr = alpha_rr.numpy()
    final_beta_rr = beta_rr.numpy()
    
    aleatoric = final_beta_resp/(final_alpha_resp - 1)
    epistemic = final_beta_resp/(final_lamda_resp*(final_alpha_resp - 1))
    
    aleatoric_rr = final_beta_rr/(final_alpha_rr - 1)
    epistemic_rr = final_beta_rr/(final_lamda_rr*(final_alpha_rr - 1))
    epistemic_rr = epistemic_rr.reshape(epistemic_rr.shape[0],epistemic_rr.shape[1])
    aleatoric_rr = aleatoric_rr.reshape(aleatoric_rr.shape[0],aleatoric_rr.shape[1])
    print(epistemic_rr)
    samples = np.arange(0,len(final_rr)).reshape(-1,1)
    epistemic_rr = epistemic_rr/max(epistemic_rr)
    aleatoric_rr = aleatoric_rr/max(aleatoric_rr)
    
    confd_data = np.hstack((samples,final_rr , epistemic_rr , aleatoric_rr))
    col_confd = ['Samples','Final RR Output (BrPM)' , 'Epistemic','Aleatoric']
    data_confd = pd.DataFrame(confd_data , columns = col_confd)    


Mean Absolute Error average wise for confd is: 2.3588727657350006
Root Mean Square Error average wise for confd is: 2.999750962348168
Mean Absolute Error instantaneous wise for confd is: 2.8022672296022826
Root Mean Square Error instantaneous wise for confd is: 3.5294598913162827
[[14.274972 ]
 [14.683477 ]
 [14.961156 ]
 [14.140909 ]
 [16.739557 ]
 [18.088484 ]
 [14.410079 ]
 [18.088484 ]
 [14.241357 ]
 [15.547629 ]
 [15.11925  ]
 [14.87392  ]
 [15.260941 ]
 [18.088484 ]
 [15.332198 ]
 [17.01018  ]
 [14.191058 ]
 [15.101618 ]
 [14.174325 ]
 [14.20781  ]
 [18.088484 ]
 [14.157611 ]
 [18.088484 ]
 [14.891333 ]
 [17.049135 ]
 [14.157611 ]
 [18.129492 ]
 [18.088484 ]
 [18.129492 ]
 [18.088484 ]
 [18.088484 ]
 [18.088484 ]
 [18.088484 ]
 [18.088484 ]
 [14.157611 ]
 [18.088484 ]
 [14.978652 ]
 [18.088484 ]
 [18.088484 ]
 [15.619997 ]
 [14.157611 ]
 [18.088484 ]
 [14.157611 ]
 [14.20781  ]
 [15.013702 ]
 [18.088484 ]
 [14.157611 ]
 [15.656284 ]
 [14.140909 ]
 [14.157611 ]
 [14.258157 ]
 [14.

In [17]:
#Mean Absolute Error average wise for confc is: 2.9281539498774567
#Root Mean Square Error average wise for confc is: 3.62999285258038

In [18]:
if input_conf == 'confc' or input_conf == 'confe' or input_conf == 'confd' or input_conf == 'conff':
    no_of_samples = 32*4
    x_unc = np.linspace(start = 0,stop = no_of_samples, num = no_of_samples)
    layout_epistemic = go.Layout(
    title = "Plotting Respiratory Waveform with Epistemic Uncertainty for "+ input_conf.upper(),
    yaxis = dict(
        title = 'Amplitude' 
    ),
    xaxis = dict(
        title = 'samples'
    )
    )

    layout_aleatoric = go.Layout(
    title = "Plotting Respiratory Waveform with Aleatoric Uncertainty for "+ input_conf.upper(),
    yaxis = dict(
        title = 'Amplitude' 
    ),
    xaxis = dict(
        title = 'samples'
    )
    )

    def update_plot(signals):
        data = []
            # Reference ECG trace
        trace_epistemic = go.Scatter(
            x = x_unc,
            y = final_output_resp_sig[signals], 
            mode = 'lines',
            name = 'Respiration with Epistemic',
                    line = dict(
                    shape = 'spline',
                    color = 'green'
                     ),
                error_y=dict(
                    type='data', # value of error bar given in data coordinates
                    array=epistemic[signals],
                    visible=True,
                    color='black',
                thickness=3,
                width=5)
                )

        trace_aleatoric = go.Scatter(
                     x = x_unc,
                     y = final_output_resp_sig[signals], 
                    mode = 'lines',
                     name = 'Respiration with Aleatoric',
                     line = dict(
                         shape = 'spline',
                         color = 'green'
                     ),
                error_y=dict(
                    type='data', # value of error bar given in data coordinates
                    array=aleatoric[signals],
                    visible=True,
                    color='black',
                thickness=3,
                width=5)
                )
        fig_epistemic = go.Figure(data = [trace_epistemic],layout = layout_epistemic)
        py.offline.iplot(fig_epistemic)
        fig_aleatoric = go.Figure(data = [trace_aleatoric],layout = layout_aleatoric)
        py.offline.iplot(fig_aleatoric)
signals_epsitemic = widgets.IntSlider(min = 0,max = len(final_output_resp_sig), value = 0, description = 'Record_no:')
widgets.interactive(update_plot, signals = signals_epsitemic)
signals_aleatoric = widgets.IntSlider(min = 0,max = len(final_output_resp_sig), value = 0, description = 'Record_no:')
widgets.interactive(update_plot, signals = signals_aleatoric)


interactive(children=(IntSlider(value=0, description='Record_no:', max=814), Output()), _dom_classes=('widget-…

In [19]:
if input_conf == 'confb':
    fig1 = px.scatter(data_confb,x = 'Samples', y="Final RR Output (BrPM)",title="Epistemic Uncertainty Distribution for "+input_conf.upper(),
                     color="Epistemic", color_continuous_scale=px.colors.sequential.Viridis)

    fig2 = px.scatter(data_confb,x = 'Samples', y="Final RR Output (BrPM)",title="Aleatoric Uncertainty Distribution for "+input_conf.upper(),
                     color="Aleatoric", color_continuous_scale=px.colors.sequential.Plasma)

    fig1.show()
    fig2.show()

if input_conf == 'confa':
    fig1 = px.scatter(data_confa,x = 'Samples', y="Final RR Output (BrPM)",title="Epistemic Uncertainty Distribution for "+input_conf.upper(),
                     color="Epistemic", color_continuous_scale=px.colors.sequential.Viridis)

    fig2 = px.scatter(data_confa, x = 'Samples',y="Final RR Output (BrPM)",title="Aleatoric Uncertainty Distribution for "+input_conf.upper(),
                     color="Aleatoric", color_continuous_scale=px.colors.sequential.Plasma)

    fig1.show()
    fig2.show()    

if input_conf == 'confd':
    fig1 = px.scatter(data_confd,x = 'Samples', y="Final RR Output (BrPM)",title="Epistemic Uncertainty Distribution for "+input_conf.upper(),
                     color="Epistemic", color_continuous_scale=px.colors.sequential.Viridis)

    fig2 = px.scatter(data_confd, x = 'Samples',y="Final RR Output (BrPM)",title="Aleatoric Uncertainty Distribution for "+input_conf.upper(),
                     color="Aleatoric", color_continuous_scale=px.colors.sequential.Plasma)

    fig1.show()
    fig2.show() 

if input_conf == 'conff':
    fig1 = px.scatter(data_conff,x = 'Samples', y="Final RR Output (BrPM)",title="Epistemic Uncertainty Distribution for "+input_conf.upper(),
                     color="Epistemic", color_continuous_scale=px.colors.sequential.Viridis)

    fig2 = px.scatter(data_conff, x = 'Samples',y="Final RR Output (BrPM)",title="Aleatoric Uncertainty Distribution for "+input_conf.upper(),
                     color="Aleatoric", color_continuous_scale=px.colors.sequential.Plasma)

    fig1.show()
    fig2.show() 